# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from helpers.index_manager import IndexManager
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.dataloader_custom_functions import collate_fn

# Loading data

In [2]:
df_train = pd.read_csv('../data/train-leave2.csv')
df_val = pd.read_csv('../data/val-leave2.csv')
df_test = pd.read_csv('../data/test-leave2.csv')
index_manager = IndexManager()
index_manager.load('../data/index.pkl')

# Creating datasets

In [7]:
train_dataset = NCFDataset(df_train)
val_dataset = NCFDataset(df_val)

dataloader_params = {
    'batch_size': 2**12,
    'num_workers': 4,
    'persistent_workers': True,
    'prefetch_factor': 4,
    'collate_fn': collate_fn,
}

train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
eval_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [8]:
params = {'factors': 8, 'mlp_user_item_dim': 128, 'learning_rate': 0.001, 'epochs': 5, 'optimizer': 'adagrad', 'dropout': 0.0, 'weight_decay': 0.0001, 'loss_fn': 'mse'}

unique_users = index_manager.get_indexed_values('user_id')
unique_items = index_manager.get_indexed_values('item_id')

model = NCFRecommender(unique_users, unique_items, **params)
model.fit(train_dataloader, eval_dataloader)

Epoch 1/5
Train loss: 0.138583, Validation loss: 0.141110
Epoch 2/5
Train loss: 0.105502, Validation loss: 0.154581
Epoch 3/5
Train loss: 0.039268, Validation loss: 0.167338
Epoch 4/5
Train loss: 0.032350, Validation loss: 0.168563
Early stopping triggered after 4 epochs
Training completed!


# Evaluating model

In [9]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing 1 of 130372 users... (0.00%)
Memory usage: 0.300048828125 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 130372 users... (0.10%)
Memory usage: 0.1865234375 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 130372 users... (0.21%)
Memory usage: 0.201416015625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 130372 users... (0.33%)
Memory usage: 0.216552734375 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 170 to 187
Increased item batch size from 1363 to 1499
Processing 595 of 130372 users... (0.46%)
Memory usage: 0.234619140625 . Increasing batch size with increasing r

In [10]:
metrics

{'Hit Ratio@10': 0.00810756910993158,
 'NDCG@10': 0.003487007847869748,
 'Recall@10': 0.00810756910993158}